# Hadoop Streaming

Usually MapReduce jobs are written in Java. Nevertheless, Hadoop has a feature called somewhat misleadingly [Hadoop Streaming](https://hadoop.apache.org/docs/stable/hadoop-streaming/HadoopStreaming.html) which enables one to use Python or any other script language such as `shell` for developing mappers and reducers.

# Writing Some Code

First, we need to code our mapper. In case of Hadoop Streaming, a mapper is a script which gets some text from the standard input until the EOF and produces some text line by line to the standard output. For example, it can be like the following file:

In [1]:
!cat mapper.py

#!/usr/bin/python3

counter = 0
while True:
    try:
        counter += 1
        input()
    except EOFError:
        break
print(counter)


Mind the first line (so-called [shebang](https://en.wikipedia.org/wiki/Shebang_(Unix))) - it's very important to keep it in place, since Hadoop doesn't know where your favourite Python executable is located. It could even be `#!/usr/bin/perl` or `#!/usr/bin/bash` as well.

This script does nothing interesting, it simply goes through the file line by line until EOF and counts lines. Then it prints the total number of lines in a file.

Of course, you can code anything more complicated: import additional packages, define functions, classes, and so forth.

The reducer looks similar since generally it does the same trick: goes through the lines of the standard input and prints something to the standard output. The main difference is that it has the output of the mapper as it's input.

In [2]:
!cat reducer.py

#!/usr/bin/python3

counter = 0
while True:
    try:
        line = input()
    except EOFError:
        break
    counter += int(line)
print(counter)


This reducer sums integer values (which are the line counts produced by the mapper).

# Pushing your code to the cluster

Hadoop lives on a cluster, and your MapReduce jobs will run on the cluster too. Hadoop __can't__ execute any code from a local machine directly. That means we need to put out code to HDFS somehow.

In [3]:
!hdfs dfs -put mapper.py code
!hdfs dfs -put reducer.py code

Remember to do that whenever your want to update your MapReduce jobs!

# Running MapReduce

We will use the `mapred streaming` command for running our Hadoop Streaming job. The description of parameters follows:
* files - here we put a comma-separated list of our source code files __on HDFS__. In case of Python they are simple Python scripts but in case of Java they would be `jar` files
* input - a file on HDFS to input to the mapper
* output - some location to HDFS where to put the results (the output of the reducer)
* mapper - the name of the mapper script
* reducer - the name of the reducer script

Let magic happen!

In [4]:
!mapred streaming \
    -files code/mapper.py,code/reducer.py \
    -input data/yelp_academic_dataset_review.json \
    -output data/result \
    -mapper mapper.py \
    -reducer reducer.py

2020-11-02 00:40:31,456 INFO impl.MetricsConfig: Loaded properties from hadoop-metrics2.properties


2020-11-02 00:40:31,538 INFO impl.MetricsSystemImpl: Scheduled Metric snapshot period at 10 second(s).


2020-11-02 00:40:31,538 INFO impl.MetricsSystemImpl: JobTracker metrics system started
2020-11-02 00:40:31,551 WARN impl.MetricsSystemImpl: JobTracker metrics system already initialized!


2020-11-02 00:40:31,727 INFO mapred.FileInputFormat: Total input files to process : 1
2020-11-02 00:40:31,756 INFO mapreduce.JobSubmitter: number of splits:189


2020-11-02 00:40:32,062 INFO mapreduce.JobSubmitter: Submitting tokens for job: job_local1714646336_0001
2020-11-02 00:40:32,063 INFO mapreduce.JobSubmitter: Executing with tokens: []


...

41:11,780 INFO reduce.InMemoryMapOutput: Read 11 bytes from map-output for attempt_local1714646336_0001_m_000078_0
2020-11-02 00:41:11,780 INFO reduce.MergeManagerImpl: closeInMemoryFile -> map-output of size: 11, inMemoryMapOutputs.size() -> 164, commitMemory -> 1793, usedMemory ->1804
2020-11-02 00:41:11,780 INFO reduce.LocalFetcher: localfetcher#1 about to shuffle output of map attempt_local1714646336_0001_m_000181_0 decomp: 11 len: 15 to MEMORY
2020-11-02 00:41:11,780 INFO reduce.InMemoryMapOutput: Read 11 bytes from map-output for attempt_local1714646336_0001_m_000181_0
2020-11-02 00:41:11,780 INFO reduce.MergeManagerImpl: closeInMemoryFile -> map-output of size: 11, inMemoryMapOutputs.size() -> 165, commitMemory -> 1804, usedMemory ->1815
2020-11-02 00:41:11,781 INFO reduce.LocalFetcher: localfetcher#1 about to shuffle output of map attempt_local1714646336_0001_m_000034_0 decomp: 11 len: 15 to MEMORY
2020-11-02 00:41:11,781 INFO reduce.InMemoryMapOutput: Read 11 bytes from map-ou

emoryFile -> map-output of size: 11, inMemoryMapOutputs.size() -> 187, commitMemory -> 2046, usedMemory ->2057
2020-11-02 00:41:11,851 INFO reduce.LocalFetcher: localfetcher#1 about to shuffle output of map attempt_local1714646336_0001_m_000085_0 decomp: 11 len: 15 to MEMORY
2020-11-02 00:41:11,851 INFO reduce.InMemoryMapOutput: Read 11 bytes from map-output for attempt_local1714646336_0001_m_000085_0
2020-11-02 00:41:11,851 INFO reduce.MergeManagerImpl: closeInMemoryFile -> map-output of size: 11, inMemoryMapOutputs.size() -> 188, commitMemory -> 2057, usedMemory ->2068
2020-11-02 00:41:11,852 INFO reduce.LocalFetcher: localfetcher#1 about to shuffle output of map attempt_local1714646336_0001_m_000136_0 decomp: 11 len: 15 to MEMORY
2020-11-02 00:41:11,852 INFO reduce.InMemoryMapOutput: Read 11 bytes from map-output for attempt_local1714646336_0001_m_000136_0
2020-11-02 00:41:11,852 INFO reduce.MergeManagerImpl: closeInMemoryFile -> map-output of size: 11, inMemoryMapOutputs.size() -> 

2020-11-02 00:41:11,919 INFO output.FileOutputCommitter: Saved output of task 'attempt_local1714646336_0001_r_000000_0' to file:/workdir/boris/projects/big-data-exercises/hadoop-examples/data/result
2020-11-02 00:41:11,920 INFO mapred.LocalJobRunner: Records R/W=189/1 > reduce
2020-11-02 00:41:11,920 INFO mapred.Task: Task 'attempt_local1714646336_0001_r_000000_0' done.
2020-11-02 00:41:11,921 INFO mapred.Task: Final Counters for attempt_local1714646336_0001_r_000000_0: Counters: 24
	File System Counters
		FILE: Number of bytes read=6378222896
		FILE: Number of bytes written=803984
		FILE: Number of read operations=0
		FILE: Number of large read operations=0
		FILE: Number of write operations=0
	Map-Reduce Framework
		Combine input records=0
		Combine output records=0
		Reduce input groups=177
		Reduce shuffle bytes=2835
		Reduce input records=189
		Reduce output records=1
		Spilled Records=189
		Shuffled Maps =189
		Failed Shuffles=0
		Merged Map outputs=189
		GC time elapsed (ms)=0
	

2020-11-02 00:41:12,237 INFO mapreduce.Job:  map 100% reduce 100%
2020-11-02 00:41:12,238 INFO mapreduce.Job: Job job_local1714646336_0001 completed successfully
2020-11-02 00:41:12,285 INFO mapreduce.Job: Counters: 30
	File System Counters
		FILE: Number of bytes read=613863244327
		FILE: Number of bytes written=151595366
		FILE: Number of read operations=0
		FILE: Number of large read operations=0
		FILE: Number of write operations=0
	Map-Reduce Framework
		Map input records=8021122
		Map output records=189
		Map output bytes=1323
		Map output materialized bytes=2835
		Input split bytes=29106
		Combine input records=0
		Combine output records=0
		Reduce input groups=177
		Reduce shuffle bytes=2835
		Reduce input records=189
		Reduce output records=1
		Spilled Records=378
		Shuffled Maps =189
		Failed Shuffles=0
		Merged Map outputs=189
	

	GC time elapsed (ms)=585
		Total committed heap usage (bytes)=625454612480
	Shuffle Errors
		BAD_ID=0
		CONNECTION=0
		IO_ERROR=0
		WRONG_LENGTH=0
		WRONG_MAP=0
		WRONG_REDUCE=0
	File Input Format Counters 
		Bytes Read=6377293852
	File Output Format Counters 
		Bytes Written=21
2020-11-02 00:41:12,286 INFO streaming.StreamJob: Output directory: data/result


Let's see the results:

In [5]:
!hdfs dfs -ls data/result

Found 2 items


-rw-r--r--   1 boris atg          0 2020-11-02 00:41 data/result/_SUCCESS
-rw-r--r--   1 boris atg          9 2020-11-02 00:41 data/result/part-00000


Mind the file \_SUCCESS. It appears only when a MapReduce job finished successfully. Since the reducer outputs a file, here it is - `part-00000`

In [6]:
!hdfs dfs -cat data/result/part-00000

8021311	


This is the number of lines in our input file.

# Multiple Parts

Let's run the same MapReduce job but without a reducer (then it's considered to act as identity and print exactly what it reads from it's input):

In [7]:
!mapred streaming \
    -files code/mapper.py,code/reducer.py \
    -input data/yelp_academic_dataset_review.json \
    -output data/multi_result \
    -mapper mapper.py

2020-11-02 00:41:46,244 INFO impl.MetricsConfig: Loaded properties from hadoop-metrics2.properties


2020-11-02 00:41:46,337 INFO impl.MetricsSystemImpl: Scheduled Metric snapshot period at 10 second(s).
2020-11-02 00:41:46,337 INFO impl.MetricsSystemImpl: JobTracker metrics system started
2020-11-02 00:41:46,348 WARN impl.MetricsSystemImpl: JobTracker metrics system already initialized!


2020-11-02 00:41:46,560 INFO mapred.FileInputFormat: Total input files to process : 1
2020-11-02 00:41:46,580 INFO mapreduce.JobSubmitter: number of splits:189


2020-11-02 00:41:46,707 INFO mapreduce.JobSubmitter: Submitting tokens for job: job_local1141963703_0001
2020-11-02 00:41:46,707 INFO mapreduce.JobSubmitter: Executing with tokens: []


2020-11-02 00:41:46,892 INFO mapred.LocalDistributedCacheManager: Localized file:/workdir/boris/projects/big-data-exercises/hadoop-examples/code/mapper.py as file:/tmp/hadoop-boris/mapred/local/job_local1141963703_0001_cdda79bb-fd47-453e-9809-636b111a8614/mapper.py


...

128, commitMemory -> 1397, usedMemory ->1408
2020-11-02 00:42:26,562 INFO reduce.LocalFetcher: localfetcher#1 about to shuffle output of map attempt_local1141963703_0001_m_000013_0 decomp: 11 len: 15 to MEMORY
2020-11-02 00:42:26,562 INFO reduce.InMemoryMapOutput: Read 11 bytes from map-output for attempt_local1141963703_0001_m_000013_0
2020-11-02 00:42:26,562 INFO reduce.MergeManagerImpl: closeInMemoryFile -> map-output of size: 11, inMemoryMapOutputs.size() -> 129, commitMemory -> 1408, usedMemory ->1419
2020-11-02 00:42:26,563 INFO reduce.LocalFetcher: localfetcher#1 about to shuffle output of map attempt_local1141963703_0001_m_000064_0 decomp: 11 len: 15 to MEMORY
2020-11-02 00:42:26,563 INFO reduce.InMemoryMapOutput: Read 11 bytes from map-output for attempt_local1141963703_0001_m_000064_0
2020-11-02 00:42:26,563 INFO reduce.MergeManagerImpl: closeInMemoryFile -> map-output of size: 11, inMemoryMapOutputs.size() -> 130, commitMemory -> 1419, usedMemory ->1430
2020-11-02 00:42:26,5

02 00:42:26,630 INFO reduce.InMemoryMapOutput: Read 11 bytes from map-output for attempt_local1141963703_0001_m_000070_0
2020-11-02 00:42:26,630 INFO reduce.MergeManagerImpl: closeInMemoryFile -> map-output of size: 11, inMemoryMapOutputs.size() -> 152, commitMemory -> 1661, usedMemory ->1672
2020-11-02 00:42:26,631 INFO reduce.LocalFetcher: localfetcher#1 about to shuffle output of map attempt_local1141963703_0001_m_000121_0 decomp: 11 len: 15 to MEMORY
2020-11-02 00:42:26,632 INFO reduce.InMemoryMapOutput: Read 11 bytes from map-output for attempt_local1141963703_0001_m_000121_0
2020-11-02 00:42:26,632 INFO reduce.MergeManagerImpl: closeInMemoryFile -> map-output of size: 11, inMemoryMapOutputs.size() -> 153, commitMemory -> 1672, usedMemory ->1683
2020-11-02 00:42:26,632 INFO reduce.LocalFetcher: localfetcher#1 about to shuffle output of map attempt_local1141963703_0001_m_000172_0 decomp: 11 len: 15 to MEMORY
2020-11-02 00:42:26,632 INFO reduce.InMemoryMapOutput: Read 11 bytes from 

apOutputs.size() -> 175, commitMemory -> 1914, usedMemory ->1925
2020-11-02 00:42:26,700 INFO reduce.LocalFetcher: localfetcher#1 about to shuffle output of map attempt_local1141963703_0001_m_000184_0 decomp: 11 len: 15 to MEMORY
2020-11-02 00:42:26,700 INFO reduce.InMemoryMapOutput: Read 11 bytes from map-output for attempt_local1141963703_0001_m_000184_0
2020-11-02 00:42:26,700 INFO reduce.MergeManagerImpl: closeInMemoryFile -> map-output of size: 11, inMemoryMapOutputs.size() -> 176, commitMemory -> 1925, usedMemory ->1936
2020-11-02 00:42:26,700 INFO reduce.LocalFetcher: localfetcher#1 about to shuffle output of map attempt_local1141963703_0001_m_000179_0 decomp: 11 len: 15 to MEMORY
2020-11-02 00:42:26,700 INFO reduce.InMemoryMapOutput: Read 11 bytes from map-output for attempt_local1141963703_0001_m_000179_0
2020-11-02 00:42:26,700 INFO reduce.MergeManagerImpl: closeInMemoryFile -> map-output of size: 11, inMemoryMapOutputs.size() -> 177, commitMemory -> 1936, usedMemory ->1947
2

2020-11-02 00:42:27,643 INFO mapreduce.Job:  map 100% reduce 100%
2020-11-02 00:42:27,644 INFO mapreduce.Job: Job job_local1141963703_0001 completed successfully


2020-11-02 00:42:27,693 INFO mapreduce.Job: Counters: 30
	File System Counters
		FILE: Number of bytes read=613863244327
		FILE: Number of bytes written=151279008
		FILE: Number of read operations=0
		FILE: Number of large read operations=0
		FILE: Number of write operations=0
	Map-Reduce Framework
		Map input records=8021122
		Map output records=189
		Map output bytes=1323
		Map output materialized bytes=2835
		Input split bytes=29106
		Combine input records=0
		Combine output records=0
		Reduce input groups=177
		Reduce shuffle bytes=2835
		Reduce input records=189
		Reduce output records=189
		Spilled Records=378
		Shuffled Maps =189
		Failed Shuffles=0
		Merged Map outputs=189
		GC time elapsed (ms)=674
		Total committed heap usage (bytes)=793306464256
	Shuffle Errors
		BAD_ID=0
		CONNECTION=0
		IO_ERROR=0
		WRONG_LENGTH=0
		WRONG_MAP=0
		WRONG_REDUCE=0
	File Input Format Counters 
		Bytes Read=6377293852
	File Output Format Counters 
		Bytes Written=1343
2020-11-02 00:42:27,693 IN

In [8]:
!hdfs dfs -ls data/multi_result

Found 2 items


-rw-r--r--   1 boris atg          0 2020-11-02 00:42 data/multi_result/_SUCCESS
-rw-r--r--   1 boris atg       1323 2020-11-02 00:42 data/multi_result/part-00000


Although there is only one resulting file, it has several lines:

In [9]:
!hdfs dfs -cat data/multi_result/part-00000

20952	
40877	
41193	
41213	
41226	
41240	
41255	
41257	
41260	
41266	
41279	
41280	
41295	
41306	
41317	
41331	
41351	
41352	
41355	
41374	
41386	
41395	
41404	
41404	
41409	
41414	
41420	
41442	
41444	
41455	
41462	
41488	
41504	
41517	
41530	
41535	
41563	
41567	
41570	
41591	
41601	
41604	
41608	
41615	
41639	
41650	
41660	
41678	
41678	
41705	
41726	
41731	
41732	
41734	
41741	
41751	
41755	
41770	
41786	
41787	
41790	
41811	
41840	
41863	
41873	
41887	
41898	
41922	
41922	
41979	
41995	
42002	
42074	
42101	
42108	
42141	
42143	
42157	
42224	
42256	
42260	
42305	
42323	
42324	
42367	
42502	
42516	
42554	
42556	
42593	
42656	
42695	
42699	
42717	
42748	
42792	
42800	
42825	
42931	
42962	
42978	
42991	
43068	
43068	
43070	
43077	
43089	
43112	
43122	
43124	
43135	
43137	
43140	
43151	
43175	
43175	
43175	
43185	
43198	
43199	
43232	
43237	
43239	
43244	
43244	
43246	
43247	
43268	
43269	
43278	
43279	
43285	
43294	
43298	
43298	
43304	
43307	
43309	
43327	
43328	
43335	
43343	
43355	

That happened because our mapper was run in parallel. You can wonder why it was slow then. The answer is quite simple - Python is an interpreted language, so it would be faster if we were using C++ or Java for our mappers and reducers.

# Do It Yourself
* code a mapper for counting words or characters, not lines
* code a reducer to count lines, words, and symbols in one job
* code a mapper to sum over the `compliment_count` field value